# Use Hintereisferner for real glacier example of flowline

In [ ]:
import geoviews as gv
gv.extension('matplotlib')
import salem
from salem import gis, wgs84
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm as colormap
from matplotlib.patches import Rectangle, Polygon, FancyArrowPatch
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import Size, Divider
import mpl_toolkits
import shapely.geometry as shpg
import seaborn as sns
import numpy as np
from oggm import tasks
from oggm import workflow, utils
from oggm import graphics
from oggm import cfg
cfg.initialize()

# help function for text

In [ ]:
def set_text(ax, text, alignment='left', facecolor=[0.5,0.5,0.5], fontsize=12):
    ax.text(0, 0, text,
            fontsize=fontsize,
            verticalalignment='center', horizontalalignment=alignment)

    ax.set_ylim([-1, 1])

    if alignment == 'left':
        ax.set_xlim([0, 1])
    elif alignment == 'center':
        ax.set_xlim([-1, 1])

    if type(ax.axis) == mpl_toolkits.axes_grid1.mpl_axes.Axes.AxisDict:
        for key, spine in ax.axis.items():
                spine.major_ticks.set_visible(False)
                spine.minor_ticks.set_visible(False)
                spine.line.set_visible(False)
    else:
        ax.axis('off')

    ax.set_yticklabels([])
    ax.set_xticklabels([])

    ax.set_facecolor(facecolor)

# Import Glacier and compute centerline with widths

In [ ]:
cfg.PATHS['working_dir'] = 'Hintereisferner'
rgi_ids = ['RGI60-11.00897']
gdirs = workflow.init_glacier_regions(rgi_ids, from_prepro_level=2, prepro_border=10)

In [ ]:
workflow.execute_entity_task(tasks.glacier_masks, gdirs);
# only calculate one flowline
cfg.PARAMS['use_multiple_flowlines'] = False

list_talks = [
         tasks.compute_centerlines,
         tasks.initialize_flowlines,
         tasks.compute_downstream_line,
         tasks.catchment_area,
         tasks.catchment_width_geom,
         tasks.catchment_width_correction,
         tasks.compute_downstream_bedshape
         ]
for task in list_talks:
    # The order matters!
    workflow.execute_entity_task(task, gdirs)

## Define colors/linewidth for plots

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
glacier_color = list(colors[9]) + [.5]
flowline_color = list(colors[3]) + [1.]
topo_color = list(colors[7]) + [.8]
outline_color = [0., 0., 0., 1.] #list(colors[0]) + [1.]
numeric_grid_color = [0., 0., 0., 1.]

In [ ]:
centerline_linewidth = 2.5
width_linewidth = 2.
outline_linewidth = 2.
head_marker = 'o'
head_marker_size = 150
tail_marker = 'X'
tail_marker_size = 150
numeric_grid_marker = '.'
numeric_grid_marker_size = 130

## Plot for flowline on real glacier

choose colors

In [ ]:
figsize=(20,10)
gdir = gdirs[0]

fig = plt.figure(figsize=figsize)#, facecolor='white')
ax = fig.add_subplot(111)

def plot_real_glacier(ax, head_marker_size = 150, tail_marker_size = 150, numeric_grid_marker_size = 130):
    # initiate a salem Map
    smap = salem.Map(gdirs[0].grid, countries=False,
                     nx=gdirs[0].grid.nx)

    # remove grid
    smap.set_lonlat_contours(interval=0)

    # variable to count how many lines were drawn
    nr_lines = 0

    # Plot topo as contour
    with utils.ncDataset(gdir.get_filepath('gridded_data')) as nc:
        topo = nc.variables['topo'][:]
    smap.set_contour(topo, levels=25, linestyles='dashed', colors=[topo_color])

    # get reference grid
    crs = gdir.grid.center_grid

    # Plot Centerline
    line = gdir.read_pickle('downstream_line')['full_line']
    smap.set_geometry(line, crs=crs, color=flowline_color,
                      linewidth=centerline_linewidth, zorder=50)
    nr_lines += 0

    # plot widths and points for numeric grid
    cls = gdir.read_pickle('inversion_flowlines')[::-1]
    for i, (wl, wi) in enumerate(zip(cls[0].geometrical_widths, cls[0].widths)):
        if (i) % 4 == 0:
            col = flowline_color if np.isfinite(wi) else 'grey'
            for w in wl:
                smap.set_geometry(w, crs=crs, color=flowline_color,
                                  linewidth=width_linewidth, zorder=50)
                nr_lines += 1

    # add points for numeric grid
    (x_numeric_grid, y_numeric_grid) = line = gdir.read_pickle('downstream_line')['full_line'].xy
    for i in range(len(x_numeric_grid)):
        if i == 0:
            smap.set_geometry(shpg.Point(x_numeric_grid[i], y_numeric_grid[i]),
                              crs=crs,
                              marker=head_marker,
                              markersize=head_marker_size,
                              color=flowline_color,
                              zorder=100)
        elif i == len(x_numeric_grid) - 1:
            smap.set_geometry(shpg.Point(x_numeric_grid[i], y_numeric_grid[i]),
                              crs=crs,
                              marker=tail_marker,
                              markersize=tail_marker_size,
                              color=flowline_color,
                              zorder=100)
        elif (i) % 4 == 0:
            smap.set_geometry(shpg.Point(x_numeric_grid[i], y_numeric_grid[i]),
                              crs=crs,
                              marker=numeric_grid_marker,
                              markersize=numeric_grid_marker_size,
                              color=numeric_grid_color,
                              zorder=100)

    # Plot boundaries
    geom = gdir.read_pickle('geometries')
    poly_hr = geom['polygon_hr']
    smap.set_geometry(poly_hr, crs=crs, fc=glacier_color, ec=outline_color,
                      zorder=1, lw=0., ls='-')

    out = smap.plot(ax)

    # use the patch to clip contour plot
    for collection in out['contour'][0].collections:
        collection.set_clip_path(ax.patches[0])

    # clip width lines and set outline style
    for i, line in enumerate(ax.lines):
        # do nothing on centerline
        if i == 0:
            pass
        # clip width lines
        else:
            line.set_clip_path(ax.patches[0])

        # set outline style
        if i > nr_lines:
            line.set_lw(outline_linewidth)
            line.set_color(outline_color)

    # switch grid and ax off
    plt.axis('off');

plot_real_glacier(ax)
fig.savefig('flowline_4.png',format='png',bbox_inches='tight',dpi=300)

# Numeric Domain

In [ ]:


figsize=(15,10)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

def plot_numeric_glacier(ax, fontsize=25, head_marker_size = 150, tail_marker_size = 150, numeric_grid_marker_size = 130):
    distance_left_right = 2
    glacier_cells = 5
    widths = [4, 3, 6, 5, 2]
    cells_at_start = 1
    cells_middle = 3
    cells_at_end = 3
    total_cell_number = cells_at_start + distance_left_right + cells_middle + distance_left_right + cells_at_end
    start_index_middle = cells_at_start + distance_left_right + 1

    x = np.arange(total_cell_number)
    # draw coordination system
    # y-axis
    plt.annotate(text='',
                 xy=(-1,11), 
                 xytext=(-1,-11),
                 arrowprops=dict(arrowstyle='->',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(-1.2,11.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])
    # x-axis
    plt.annotate(text='',
                 xy=(-1,0), 
                 xytext=(total_cell_number ,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(total_cell_number + 0.2,0.2,'x',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])

    # plot centerline
    plt.plot([x[0], x[0] + 0.5],
             [0., 0.],
             '-',
             color=flowline_color,
             linewidth=centerline_linewidth)

    # add dashed line
    plt.plot([x[1] - 0.5, x[2] + 0.5],
             [0,0],
             '--',
             color=flowline_color,
             linewidth=centerline_linewidth)

    # plot centerline
    plt.plot([x[3] - 0.5, x[5] + 0.5],
             [0, 0],
             '-',
             color=flowline_color,
             linewidth=centerline_linewidth)

    # plot dashed line
    plt.plot([x[6] - 0.5, x[7] + 0.5],
             [0,0],
             '--',
             color=flowline_color,
             linewidth=centerline_linewidth)

    # plot centerline
    plt.plot([x[8] - 0.5, x[10]],
             [0, 0],
             '-',
             color=flowline_color,
             linewidth=centerline_linewidth)


    numeric_grid_labels = [r'$1$',
                           r'$k - \frac{3}{2}$',
                           r'$k - 1$',
                           r'$k - \frac{1}{2}$',
                           r'$k$',
                           r'$k + \frac{1}{2}$',
                           r'$k + 1$',
                           r'$k + \frac{3}{2}$',
                           r'$trm$',
                           r'$nx$',
                          ]
    numeric_grid_x = [0, 
                      2.5, #stag
                      3, 
                      3.5, #stag
                      4, 
                      4.5, #stag
                      5, 
                      5.5, #stag
                      8, 
                      10]
    numeric_grid_y = [widths[0], # start
                      widths[1], #stag
                      widths[1], 
                      max(widths[1], widths[2]), #stag
                      widths[2], 
                      max(widths[2], widths[3]), #stag
                      widths[3], 
                      widths[3], #stag
                      widths[4],
                      0]
    stag_height = 8.1
    non_stag_height = 11
    numeric_grid_ys = [non_stag_height,
                       stag_height, 
                       non_stag_height,
                       stag_height, 
                       non_stag_height,
                       stag_height, 
                       non_stag_height,
                       stag_height,
                       non_stag_height,
                       non_stag_height]
    for i, text in enumerate(numeric_grid_labels):
        x_text = numeric_grid_x[i]
        y_text = numeric_grid_ys[i]
        ax.text(x_text,y_text,text,
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=fontsize,
                c=list(colors[7]) + [1.],
                zorder=1)
        if i not in [14, 15]:
            plt.plot([x_text, x_text],
                     [numeric_grid_y[i], y_text-0.5],
                     '-',
                     c=list(colors[7]) + [1.],
                     lw=0.8,
                     zorder=1)

    # add delta x labels with arrow and lines
    plt.annotate(text='',
                 xy=(2.5,-8.8), 
                 xytext=(3.5,-8.8),
                 arrowprops=dict(arrowstyle='<->',
                                 color=list(colors[7]) + [1.],
                                 lw=0.8),
                 )
    plt.text(3,-8.2,r'$\Delta x$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    plt.plot([2.5, 2.5], [-widths[1], -9],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)
    plt.plot([3.5, 3.5], [-widths[2], -9],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)

    plt.annotate(text='',
                 xy=(4,-8.8), 
                 xytext=(5,-8.8),
                 arrowprops=dict(arrowstyle='<->',
                                 color=list(colors[7]) + [1.],
                                 lw=0.8),
                 )
    plt.text(4.5,-8.2,r'$\Delta x$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    plt.plot([4, 4], [-widths[2], -9],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)
    plt.plot([5, 5], [-widths[3], -9],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)

    # add arrow for wk
    x_wk = x[2] - 0.5
    plt.plot([x_wk, x[3]+0.5],
             [widths[2], widths[2]],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)
    plt.plot([x_wk, x[3]+0.5],
             [-widths[2], -widths[2]],
             color=list(colors[7]) + [1.],
             lw=0.8,
             zorder=1)
    plt.annotate(text='',
                 xy=(x_wk+0.2, widths[2]), 
                 xytext=(x_wk+0.2, -widths[2]),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 zorder=1)
    plt.text(x_wk,1,r'$w_k$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])

    # plot boxes, widths and numeric grid points
    for i in range(glacier_cells):
        if i < cells_at_start:
            x_box = x[i] - 0.5
            x_width = x[i]
        elif i == 4:
            x_box = x[4 + i] - 0.5
            x_width = x[4 + i]
        else:
            x_box = x[2 + i] - 0.5
            x_width = x[2 + i]
        y_box = -widths[i]
        ax.add_patch( Rectangle((x_box, y_box), 
                                1, 2*widths[i], 
                                fc =glacier_color,  
                                ec =outline_color, 
                                lw = outline_linewidth-1,
                                zorder=2)
                     ) 

        y_width = widths[i]
        plt.plot([x_width, x_width], [y_width, -y_width], '-',
                 color=flowline_color, 
                 linewidth=width_linewidth,
                 zorder=3)
        if i != 0:
            plt.plot([x_width], [0],
                     numeric_grid_marker,
                     markersize=numeric_grid_marker_size/10,
                     color=numeric_grid_color,
                     zorder=4)

    # add numeric point at end
    plt.plot([x[-2]], [0],
             numeric_grid_marker,
             markersize=numeric_grid_marker_size/10,
             color=numeric_grid_color)

    # add marker at highest point
    plt.plot(0, 0, head_marker,
             markersize=head_marker_size / 10,
             color=flowline_color)

    # add marker at lowest point
    plt.plot(x[-1], 0,
             tail_marker,
             markersize=tail_marker_size / 10,
             color=flowline_color)

    ax.set_ylim([-11.5, 11.5])
    ax.axis('off');

plot_numeric_glacier(ax)
fig.savefig('numeric_grid_3.png',format='png',bbox_inches='tight',dpi=300)

# Cross Section

## Rectangular

In [ ]:
figsize=(8,8)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

def plot_rectangular_cross_section(ax, status=2, fontsize=25, numeric_grid_marker_size = 130):
    z_bias = 2

    # define some coordinates
    wk = 7
    h1 = 5
    h2 = 3


    # Rectangular
    rec_x_left = -3.5
    rec_x_right = 3.5

    # width
    plt.plot([rec_x_left, rec_x_right],
             [h1 + z_bias, h1 + z_bias],
             '-',
             color=flowline_color, 
             linewidth=outline_linewidth,
             zorder=3)

    # first cross section
    ax.add_patch(Rectangle((rec_x_left, z_bias), 
                            rec_x_right - rec_x_left,
                            h1, 
                            fc =glacier_color,  
                            ec =None, 
                            lw = outline_linewidth-1))

    if status > 1:
        # second cross section
        ax.add_patch(Rectangle((rec_x_left, h1 - h2 + z_bias), 
                                rec_x_right - rec_x_left,
                                h2, 
                                fc =None,  
                                ec =list(colors[7]) + [0.8],
                                fill=False,
                                hatch='/'))

        # second cross section outline
        plt.plot([rec_x_left, rec_x_right],
                 [h1 - h2 + z_bias, h1 - h2 + z_bias],
                 color=list(colors[7]) + [0.8],
                 lw = 0.8)


    # add numeric points
    plt.plot([(rec_x_left + rec_x_right) / 2], [z_bias],
             marker=numeric_grid_marker,
             markersize=numeric_grid_marker_size/10,
             color=numeric_grid_color)

    if status > 1:
        plt.plot([(rec_x_left + rec_x_right) / 2], [h1 - h2 + z_bias],
                 marker=numeric_grid_marker,
                 markersize=numeric_grid_marker_size/10,
                 color=list(colors[7]) + [0.8])


    '''
    # add wk helplines
    plt.plot([rec_x_left, rec_x_left],
             [h1 + z_bias, h1 + z_bias + 1],
             color=list(colors[7]) + [1.],
             lw=0.8)
    plt.plot([rec_x_right, rec_x_right],
             [h1 + z_bias, h1 + z_bias + 1],
             color=list(colors[7]) + [1.],
             lw=0.8)
    '''
    
    if status > 0:
        # add wk arrow
        plt.annotate(text='',
                     xy=(rec_x_left, h1 + z_bias + 0.8), 
                     xytext=(rec_x_right, h1 + z_bias + 0.8),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text((rec_x_left + rec_x_right) / 2 + 0.6,
                 h1 + z_bias + 1.1,
                 r'$w_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    # first cross section outline
    plt.plot([rec_x_left, rec_x_left, rec_x_right, rec_x_right],
             [h1 + z_bias + 1, 0 + z_bias, 0 + z_bias, h1 + z_bias + 1],
             color=outline_color,
             lw = outline_linewidth-1,
             zorder=5)

    # add z-axis
    plt.annotate(text='',
                 xy=(0,h1 + z_bias + 3), 
                 xytext=(0,0),
                 arrowprops=dict(arrowstyle='->',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(-0.2,h1 + z_bias + 3 + 0.2,'z',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])

    # add arrows for h, b and s
    x_diff_arrows = 1.5
    x_diff_first_arrow = 0.8
    x_diff_label_arrow = 0.6
    x_overlength_line = 0.1
    x_h_b_1_arrow = rec_x_left - x_diff_arrows - x_diff_first_arrow
    x_h_b_2_arrow = rec_x_left -  x_diff_first_arrow
    x_s_arrow = rec_x_left - 2 * x_diff_arrows - x_diff_first_arrow

    # add y-axis
    plt.annotate(text='',
                 xy=(x_s_arrow - x_overlength_line - 0.5,0), 
                 xytext=(rec_x_right + 1,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(rec_x_right + 1 + 0.2,0.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])
    '''
    # h1 and h2
    # bottom line h2
    plt.plot([x_h_b_2_arrow - x_overlength_line, rec_x_left],
             [(h1-h2) + z_bias, (h1-h2) + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h1
    plt.plot([x_h_b_1_arrow - x_overlength_line, rec_x_left],
             [0 + z_bias, 0 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # arrow and text h2
    plt.annotate(text='',
                 xy=(x_h_b_2_arrow, h1 + z_bias), 
                 xytext=(x_h_b_2_arrow, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_2_arrow - x_diff_label_arrow,
             h1 - h2 / 2 + z_bias,
             r'$h_{k2}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    # arrow and text h1
    plt.annotate(text='',
                 xy=(x_h_b_1_arrow, h1 + z_bias), 
                 xytext=(x_h_b_1_arrow, 0 + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_1_arrow - x_diff_label_arrow,
             h1 / 2 + z_bias,
             r'$h_{k1}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    '''
    if status > 0:
        # add arrow and text for bk1
        plt.annotate(text='',
                     xy=(x_h_b_2_arrow, z_bias), 
                     xytext=(x_h_b_2_arrow, 0),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_2_arrow - x_diff_label_arrow,
                 z_bias / 2,
                 r'$b_{k1}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_2_arrow - x_overlength_line, rec_x_left],
                 [0 + z_bias, 0 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 1:
        # add arrow and text for bk2
        plt.annotate(text='',
                     xy=(x_h_b_1_arrow, 0), 
                     xytext=(x_h_b_1_arrow, (h1-h2) + z_bias),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_1_arrow - x_diff_label_arrow,
                 (h1-h2 + z_bias) / 2,
                 r'$b_{k2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_1_arrow - x_overlength_line, rec_x_left],
                 [(h1-h2) + z_bias, (h1-h2) + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 0:
        # add arrow and text for sk
        # top line 
        plt.plot([x_s_arrow - x_overlength_line, rec_x_left],
                 [h1 + z_bias, h1 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        plt.annotate(text='',
                     xy=(x_s_arrow, 0), 
                     xytext=(x_s_arrow, h1 + z_bias),
                     arrowprops=dict(arrowstyle='<->', 
                                     color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_s_arrow - x_diff_label_arrow,
                 (h1 + z_bias) / 2,
                 r'$s_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    ax.set_ylim([-1, h1 + z_bias + 4])
    ax.set_xlim([rec_x_left - 5, rec_x_right + 2])
    ax.axis('off');

plot_rectangular_cross_section(ax, status=2)
#fig.savefig('rectangular_shape_3.png',format='png',bbox_inches='tight',dpi=300)

## Parabolic

In [ ]:
figsize=(8,8)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

def plot_parabolic_cross_section(ax, status=0, fontsize = 25, numeric_grid_marker_size = 130):
    z_bias = 2

    # define some coordinates
    wk = 7
    h1 = 5
    h2 = 3

    # Parabolic
    par_x_left = -3.5
    par_x_right = 3.5

    # width
    plt.plot([par_x_left, par_x_right],
             [h1 + z_bias, h1 + z_bias],
             '-',
             color=flowline_color, 
             linewidth=outline_linewidth,
             zorder=4)

    def get_parabolic_coordinates_Polygon(w,h,x_point, y_point):
        w_lim = w / 2
        x_relative = np.linspace(-w_lim, w_lim, 100, endpoint=True)

        Ps = 4 * h / w**2
        z = Ps * x_relative**2 + y_point
        x = x_relative + x_point
        return np.concatenate((np.expand_dims(x, axis=1),np.expand_dims(z, axis=1)), axis=1)

    def get_parabolic_coordinates_line(w,h,x_point, y_point):
        Ps = 4 * h / w**2
        w_lim = np.sqrt(4 * (h + 1) / Ps)
        x_relative = np.linspace(-w_lim/2, w_lim/2, 100, endpoint=True)


        z = Ps * x_relative**2 + y_point
        x = x_relative + x_point
        return x, z

    # first cross section
    ax.add_patch(Polygon(get_parabolic_coordinates_Polygon(wk,h1,(par_x_left + par_x_right)/ 2, 0 + z_bias),
                         fc =glacier_color,  
                         ec =None,
                         closed=False,
                         lw = outline_linewidth-1))
    if status > 1:
        # second cross section
        ax.add_patch(Polygon(get_parabolic_coordinates_Polygon(wk,h2,(par_x_left + par_x_right)/ 2, h1 - h2 + z_bias),
                             fc =glacier_color,
                             closed=False,
                             lw = outline_linewidth-1,
                             ec =list(colors[7]) + [0.8],
                             fill=False,
                             hatch='/'))

        # second cross section outline
        x, y = get_parabolic_coordinates_line(wk,h2,(par_x_left + par_x_right)/ 2, h1 - h2 + z_bias)
        plt.plot(x,
                 y,
                 color=list(colors[7]) + [0.8],
                 lw = 0.8)

    '''
    # add arrows for h1 and h2
    x_h1_label = par_x_left - 1.4
    x_h2_label = par_x_left - 0.7

    # top line h1 and h2
    plt.plot([x_h1_label - 0.1, par_x_right],
             [h1 + z_bias, h1 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h2
    plt.plot([x_h2_label - 0.1, (par_x_right + par_x_left) / 2],
             [(h1-h2) + z_bias, (h1-h2) + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h1
    plt.plot([x_h1_label - 0.1, (par_x_right + par_x_left) / 2],
             [0 + z_bias, 0 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # arrow and text h2
    plt.annotate(text='',
                 xy=(x_h2_label, h1 + z_bias), 
                 xytext=(x_h2_label, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h2_label - 0.3,
             h1 - h2 / 2 + z_bias,
             r'$h_2$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    # arrow and text h1
    plt.annotate(text='',
                 xy=(x_h1_label, h1 + z_bias), 
                 xytext=(x_h1_label, 0 + z_bias),
                 arrowprops=dict(arrowstyle='<->',
                                 color=list(colors[7]) + [1.],
                                 lw=0.8),
                 )
    plt.text(x_h1_label - 0.3,
             h1 / 2 + z_bias,
             r'$h_1$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    '''
    # points of numeric grid
    plt.plot([(par_x_left + par_x_right) / 2], 0 + z_bias,
             marker=numeric_grid_marker,
             markersize=numeric_grid_marker_size/10,
             color=numeric_grid_color)

    if status > 1:
        plt.plot([(par_x_left + par_x_right) / 2], [h1-h2 + z_bias],
                 marker=numeric_grid_marker,
                 markersize=numeric_grid_marker_size/10,
                 color=list(colors[7]) + [0.8])

    if status > 0:
        # add wk helplines
        plt.plot([par_x_left, par_x_left],
                 [h1 + z_bias, h1 + z_bias + 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)
        plt.plot([par_x_right, par_x_right],
                 [h1 + z_bias, h1 + z_bias + 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        # add wk arrow
        plt.annotate(text='',
                     xy=(par_x_left, h1 + z_bias + .8), 
                     xytext=(par_x_right, h1 + z_bias + .8),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )

        # add wk text
        plt.text((par_x_left + par_x_right) / 2 +0.6,
                 h1 + z_bias + 1.1,
                 r'$w_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    # first cross section outline
    x, y = get_parabolic_coordinates_line(wk,h1,(par_x_left + par_x_right)/ 2, 0 + z_bias)
    plt.plot(x,
             y,
             color=outline_color,
             lw = outline_linewidth-1,
             zorder=5)

    # add z-axis
    plt.annotate(text='',
                 xy=(0,h1 + z_bias + 3), 
                 xytext=(0,0),
                 arrowprops=dict(arrowstyle='->',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(-0.2,h1 + z_bias + 3 + 0.2,'z',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])
    '''
    # add y-axis
    plt.annotate(text='',
                 xy=(rec_x_left - 2 ,0), 
                 xytext=(rec_x_right + 2,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(rec_x_right + 2 + 0.2,0.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=16,
             c=list(colors[7]) + [1.])

    # add arrow and text for bed_h1
    plt.annotate(text='',
                 xy=(x_h1_label, z_bias), 
                 xytext=(x_h1_label, 0),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h1_label - 0.3,
             z_bias / 2,
             r'$b_1$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])

    # add arrow and text for bed_h2
    plt.annotate(text='',
                 xy=(x_h2_label, 0), 
                 xytext=(x_h2_label, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h2_label - 0.3,
             (h1-h2 + z_bias) / 2,
             r'$b_2$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    '''
    # add arrows for h, b and s
    x_diff_arrows = 1.5
    x_diff_first_arrow = 0.8
    x_diff_label_arrow = 0.6
    x_overlength_line = 0.1
    x_h_b_1_arrow = par_x_left - x_diff_arrows - x_diff_first_arrow
    x_h_b_2_arrow = par_x_left -  x_diff_first_arrow
    x_s_arrow = par_x_left - 2 * x_diff_arrows - x_diff_first_arrow

    # add y-axis
    plt.annotate(text='',
                 xy=(x_s_arrow - x_overlength_line - 0.5,0), 
                 xytext=(par_x_right + 1,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(par_x_right + 1 + 0.2,0.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])
    '''
    # top line h1 and h2
    plt.plot([x_s_arrow - x_overlength_line, par_x_left],
             [h1 + z_bias, h1 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h2
    plt.plot([x_h_b_2_arrow - x_overlength_line, 0],
             [(h1-h2) + z_bias, (h1-h2) + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h1
    plt.plot([x_h_b_1_arrow - x_overlength_line, 0],
             [0 + z_bias, 0 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # arrow and text h2
    plt.annotate(text='',
                 xy=(x_h_b_2_arrow, h1 + z_bias), 
                 xytext=(x_h_b_2_arrow, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_2_arrow - x_diff_label_arrow,
             h1 - h2 / 2 + z_bias,
             r'$h_{k2}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    # arrow and text h1
    plt.annotate(text='',
                 xy=(x_h_b_1_arrow, h1 + z_bias), 
                 xytext=(x_h_b_1_arrow, 0 + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_1_arrow - x_diff_label_arrow,
             h1 / 2 + z_bias,
             r'$h_{k1}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    '''
    if status > 0:
        # add arrow and text for bk1
        plt.annotate(text='',
                     xy=(x_h_b_2_arrow, z_bias), 
                     xytext=(x_h_b_2_arrow, 0),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_2_arrow - x_diff_label_arrow,
                 z_bias / 2,
                 r'$b_{k1}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_2_arrow - x_overlength_line, 0],
                 [0 + z_bias, 0 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 1:
        # add arrow and text for bk2
        plt.annotate(text='',
                     xy=(x_h_b_1_arrow, 0), 
                     xytext=(x_h_b_1_arrow, (h1-h2) + z_bias),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_1_arrow - x_diff_label_arrow,
                 (h1-h2 + z_bias) / 2,
                 r'$b_{k2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_1_arrow - x_overlength_line, 0],
                 [(h1-h2) + z_bias, (h1-h2) + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 0:
        # add arrow and text for sk
        plt.annotate(text='',
                     xy=(x_s_arrow, 0), 
                     xytext=(x_s_arrow, h1 + z_bias),
                     arrowprops=dict(arrowstyle='<->', 
                                     color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_s_arrow - x_diff_label_arrow,
                 (h1 + z_bias) / 2,
                 r'$s_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_s_arrow - x_overlength_line, par_x_left],
                 [h1 + z_bias, h1 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        # add bed formula for bk1
        plt.text(2.8,
                 1.5,
                 r'$b_{k1}+Ps_{k1}\cdot y^{2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 color=list(colors[7]) + [1.])

    if status > 1:
        # add bed formula for bk2
        plt.text(2.8,
                 3.5,
                 r'$b_{k2}+Ps_{k2}\cdot y^{2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 color=list(colors[7]) + [1.])

        # add text for Ps1>Ps2
        plt.text(2.8,
                 9,
                 r'$Ps_{k1} > Ps_{k2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 color=list(colors[7]) + [1.])

    ax.set_ylim([-1, h1 + z_bias + 4])
    ax.set_xlim([par_x_left - 5, par_x_right + 2])
    ax.axis('off');

plot_parabolic_cross_section(ax, status=2)
fig.savefig('parabolic_shape_2.pdf',format='pdf',bbox_inches='tight',dpi=300)

## Trapezoidal

In [ ]:
figsize=(8,8)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

def plot_trapezoidal_cross_section(ax, status=3, fontsize=25, numeric_grid_marker_size = 130):
    z_bias = 2

    # define some coordinates
    wk = 7
    h1 = 5
    h2 = 3
    lam = 1

    # Parabolic
    par_x_left = -3.5
    par_x_right = 3.5

    # width
    plt.plot([par_x_left, par_x_right],
             [h1 + z_bias, h1 + z_bias],
             '-',
             color=flowline_color, 
             linewidth=outline_linewidth,
             zorder=4)

    def get_trapezoidal_coordinates_Polygon(w,h,lam,x_point, z_point):
        w0 = w - lam * h
        x = x_point + np.array([-w/2, -w0/2, w0/2, w/2])
        z = z_point + np.array([h, 0, 0, h])

        return np.concatenate((np.expand_dims(x, axis=1),np.expand_dims(z, axis=1)), axis=1)

    def get_trapezoidal_coordinates_line(w,h,lam,x_point, z_point):
        w0 = w - lam * h
        x = x_point + np.array([-w/2, -w0/2, w0/2, w/2])
        z = z_point + np.array([h, 0, 0, h])
        return x, z, w0

    # first cross section
    ax.add_patch(Polygon(get_trapezoidal_coordinates_Polygon(wk,h1,lam, (par_x_left + par_x_right)/ 2, 0 + z_bias),
                         fc =glacier_color,  
                         ec =None,
                         closed=False,
                         lw = outline_linewidth-1))
    if status > 1:
        # second cross section
        ax.add_patch(Polygon(get_trapezoidal_coordinates_Polygon(wk,h2,lam,(par_x_left + par_x_right)/ 2, h1 - h2 + z_bias),
                             fc =glacier_color,
                             closed=False,
                             lw = outline_linewidth-1,
                             ec =list(colors[7]) + [0.8],
                             fill=False,
                             hatch='/'))

        # second cross section outline
        x, z, w02 = get_trapezoidal_coordinates_line(wk,h2,lam,(par_x_left + par_x_right)/ 2, h1 - h2 + z_bias)
        plt.plot(x,
                 z,
                 color=list(colors[7]) + [0.8],
                 lw = 0.8)

    '''
    # add arrows for h1 and h2
    x_h1_label = par_x_left - 1.4
    x_h2_label = par_x_left - 0.7

    # top line h1 and h2
    plt.plot([x_h1_label - 0.1, par_x_right],
             [h1 + z_bias, h1 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h2
    plt.plot([x_h2_label - 0.1, (par_x_right + par_x_left) / 2],
             [(h1-h2) + z_bias, (h1-h2) + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h1
    plt.plot([x_h1_label - 0.1, (par_x_right + par_x_left) / 2],
             [0 + z_bias, 0 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # arrow and text h2
    plt.annotate(text='',
                 xy=(x_h2_label, h1 + z_bias), 
                 xytext=(x_h2_label, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h2_label - 0.3,
             h1 - h2 / 2 + z_bias,
             r'$h_2$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    # arrow and text h1
    plt.annotate(text='',
                 xy=(x_h1_label, h1 + z_bias), 
                 xytext=(x_h1_label, 0 + z_bias),
                 arrowprops=dict(arrowstyle='<->',
                                 color=list(colors[7]) + [1.],
                                 lw=0.8),
                 )
    plt.text(x_h1_label - 0.3,
             h1 / 2 + z_bias,
             r'$h_1$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    '''

    # points of numeric grid
    plt.plot([(par_x_left + par_x_right) / 2], 0 + z_bias,
             marker=numeric_grid_marker,
             markersize=numeric_grid_marker_size/10,
             color=numeric_grid_color)
    if status > 1:
        plt.plot([(par_x_left + par_x_right) / 2], [h1-h2 + z_bias],
                 marker=numeric_grid_marker,
                 markersize=numeric_grid_marker_size/10,
                 color=list(colors[7]) + [0.8])


    if status > 0:
        # add wk helplines
        plt.plot([par_x_left, par_x_left],
                 [h1 + z_bias, h1 + z_bias + 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)
        plt.plot([par_x_right, par_x_right],
                 [h1 + z_bias, h1 + z_bias + 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        # add wk arrow
        plt.annotate(text='',
                     xy=(par_x_left, h1 + z_bias + .8), 
                     xytext=(par_x_right, h1 + z_bias + .8),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )

        # add wk text
        plt.text((par_x_left + par_x_right) / 2 +0.6,
                 h1 + z_bias + 1.1,
                 r'$w_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    # first cross section outline
    x, y, w01 = get_trapezoidal_coordinates_line(wk,h1,lam,(par_x_left + par_x_right)/ 2, 0 + z_bias)
    plt.plot(x,
             y,
             color=outline_color,
             lw = outline_linewidth-1,
             zorder=5)

    # add z-axis
    plt.annotate(text='',
                 xy=(0,h1 + z_bias + 3), 
                 xytext=(0,0),
                 arrowprops=dict(arrowstyle='->',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(-0.2,h1 + z_bias + 3 + 0.2,'z',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])

    '''
    # add y-axis
    plt.annotate(text='',
                 xy=(rec_x_left - 2 ,0), 
                 xytext=(rec_x_right + 2,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(rec_x_right + 2 + 0.2,0.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=16,
             c=list(colors[7]) + [1.])

    # add arrow and text for bed_h1
    plt.annotate(text='',
                 xy=(x_h1_label, z_bias), 
                 xytext=(x_h1_label, 0),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h1_label - 0.3,
             z_bias / 2,
             r'$b_1$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])

    # add arrow and text for bed_h2
    plt.annotate(text='',
                 xy=(x_h2_label, 0), 
                 xytext=(x_h2_label, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h2_label - 0.3,
             (h1-h2 + z_bias) / 2,
             r'$b_2$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=14,
             c=list(colors[7]) + [1.])
    '''

    # add arrows for h, b and s
    x_diff_arrows = 1.5
    x_diff_first_arrow = 0.8
    x_diff_label_arrow = 0.6
    x_overlength_line = 0.1
    x_h_b_1_arrow = par_x_left - x_diff_arrows - x_diff_first_arrow
    x_h_b_2_arrow = par_x_left -  x_diff_first_arrow
    x_s_arrow = par_x_left - 2 * x_diff_arrows - x_diff_first_arrow

    # add y-axis
    plt.annotate(text='',
                 xy=(x_s_arrow - x_overlength_line - 0.5,0), 
                 xytext=(par_x_right + 1,0),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=list(colors[7]) + [1.],
                                 lw=1),
                 zorder=1
                 )
    plt.text(par_x_right + 1 + 0.2,0.2,'y',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=list(colors[7]) + [1.])

    '''
    # top line h1 and h2
    plt.plot([x_s_arrow - x_overlength_line, par_x_left],
             [h1 + z_bias, h1 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h2
    plt.plot([x_h_b_2_arrow - x_overlength_line, 0],
             [(h1-h2) + z_bias, (h1-h2) + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # bottom line h1
    plt.plot([x_h_b_1_arrow - x_overlength_line, 0],
             [0 + z_bias, 0 + z_bias],
             color=list(colors[7]) + [1.],
             lw=0.8)
    # arrow and text h2
    plt.annotate(text='',
                 xy=(x_h_b_2_arrow, h1 + z_bias), 
                 xytext=(x_h_b_2_arrow, (h1-h2) + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_2_arrow - x_diff_label_arrow,
             h1 - h2 / 2 + z_bias,
             r'$h_{k2}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    # arrow and text h1
    plt.annotate(text='',
                 xy=(x_h_b_1_arrow, h1 + z_bias), 
                 xytext=(x_h_b_1_arrow, 0 + z_bias),
                 arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                 )
    plt.text(x_h_b_1_arrow - x_diff_label_arrow,
             h1 / 2 + z_bias,
             r'$h_{k1}$',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize,
             c=list(colors[7]) + [1.])
    '''

    if status > 0:
        # add arrow and text for bk1
        plt.annotate(text='',
                     xy=(x_h_b_2_arrow, z_bias), 
                     xytext=(x_h_b_2_arrow, 0),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_2_arrow - x_diff_label_arrow,
                 z_bias / 2,
                 r'$b_{k1}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_2_arrow - x_overlength_line, 0],
                 [0 + z_bias, 0 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 1:
        # add arrow and text for bk2
        plt.annotate(text='',
                     xy=(x_h_b_1_arrow, 0), 
                     xytext=(x_h_b_1_arrow, (h1-h2) + z_bias),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_h_b_1_arrow - x_diff_label_arrow,
                 (h1-h2 + z_bias) / 2,
                 r'$b_{k2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_h_b_1_arrow - x_overlength_line, 0],
                 [(h1-h2) + z_bias, (h1-h2) + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 0:
        # add arrow and text for sk
        plt.annotate(text='',
                     xy=(x_s_arrow, 0), 
                     xytext=(x_s_arrow, h1 + z_bias),
                     arrowprops=dict(arrowstyle='<->', 
                                     color=list(colors[7]) + [1.],lw=0.8),
                     )
        plt.text(x_s_arrow - x_diff_label_arrow,
                 (h1 + z_bias) / 2,
                 r'$s_k$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])
        plt.plot([x_s_arrow - x_overlength_line, par_x_left],
                 [h1 + z_bias, h1 + z_bias],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

    if status > 0:
        # add w01 helplines
        plt.plot([-w01/2, -w01/2],
                 [0 + z_bias, 0 + z_bias - 1.2],
                 color=list(colors[7]) + [1.],
                 lw=0.8)
        plt.plot([w01/2, w01/2],
                 [0 + z_bias, 0 + z_bias - 1.2],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        # add w01 arrow
        plt.annotate(text='',
                     xy=(-w01/2, 0 + z_bias - 1.), 
                     xytext=(w01/2, 0 + z_bias - 1.),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )

        # add w01 text
        plt.text((par_x_left + par_x_right) / 2, #+ 0.6,
                 0 + z_bias - 0.6,
                 r'$w0_{k1}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    if status > 1:
        # add w02 helplines
        plt.plot([-w02/2, -w02/2],
                 [h1 - h2 + z_bias, h1 - h2 + z_bias - 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)
        plt.plot([w02/2, w02/2],
                 [h1 - h2 + z_bias, h1 - h2 + z_bias - 1],
                 color=list(colors[7]) + [1.],
                 lw=0.8)

        # add w02 arrow
        plt.annotate(text='',
                     xy=(-w02/2, h1 - h2 + z_bias - 1.), 
                     xytext=(w02/2, h1 - h2 + z_bias - 1.),
                     arrowprops=dict(arrowstyle='<->', color=list(colors[7]) + [1.],lw=0.8),
                     )

        # add w02 text
        plt.text((par_x_left + par_x_right) / 2, #+ 0.6,
                 h1 - h2 + z_bias - 0.6,
                 r'$w0_{k2}$',
                 horizontalalignment='center',
                 verticalalignment='center',
                 fontsize=fontsize,
                 c=list(colors[7]) + [1.])

    ax.set_ylim([-1, h1 + z_bias + 4])
    ax.set_xlim([par_x_left - 5, par_x_right + 2])
    #ax.set_aspect('equal')
    ax.axis('off');

plot_trapezoidal_cross_section(ax, status=2)
fig.savefig('trapezoidal_shape_3.png',format='png',bbox_inches='tight',dpi=300)

## Put all Cross section into one figure

In [ ]:
def set_text(ax, text, alignment='left', facecolor=[1,1,1], fontsize=25):
    ax.text(0, 0, text,
            fontsize=fontsize,
            verticalalignment='center', horizontalalignment=alignment)

    ax.set_ylim([-1, 1])

    if alignment == 'left':
        ax.set_xlim([0, 1])
    elif alignment == 'center':
        ax.set_xlim([-1, 1])

    if type(ax.axis) == mpl_toolkits.axes_grid1.mpl_axes.Axes.AxisDict:
        for key, spine in ax.axis.items():
                spine.major_ticks.set_visible(False)
                spine.minor_ticks.set_visible(False)
                spine.line.set_visible(False)
    else:
        ax.axis('off')

    ax.set_yticklabels([])
    ax.set_xticklabels([])

    ax.set_facecolor(facecolor)

In [ ]:
def create_cross_section_figure(filename, status=0):
    # define heights
    title_height = 1
    separation_titel_cross_section = 0.2
    cross_section_height = 10
    separation_cross_section_equation = 0.2
    equation_height = 1

    #define widths
    cross_section_width = 10
    separation_cross_section = 0.3

    # fixed size in inch
    # along x axis                                                              x-index for locator
    horiz = [Size.Fixed(cross_section_width),                                    # 0 first cross section
             Size.Fixed(separation_cross_section),   
             Size.Fixed(cross_section_width),                                    # 2 second cross section
             Size.Fixed(separation_cross_section),   
             Size.Fixed(cross_section_width),                                    # 4 third cross section
            ]

                                                                              # y-index for locator
    vert = [Size.Fixed(equation_height),                                      # 0 equation
            Size.Fixed(separation_cross_section_equation),
            Size.Fixed(cross_section_height),                                 # 2 cross section
            Size.Fixed(separation_titel_cross_section),
            Size.Fixed(title_height),                                         # 4 title
           ]

    rect = (0., 0., 1., 1.)  # Position of the grid in the figure

    fig = plt.figure(figsize=(1, 1), facecolor='white')

    # divide the axes rectangle into grid whose size is specified by horiz * vert
    divider = Divider(fig, rect, horiz, vert, aspect=False)

    # add titles
    title_fontsize = 60
    ax = fig.subplots()
    set_text(ax, 'Rectangular', alignment='center', fontsize=title_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=0 , ny=4))
    
    ax = fig.subplots()
    set_text(ax, 'Parabolic', alignment='center', fontsize=title_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=2 , ny=4))
    
    ax = fig.subplots()
    set_text(ax, 'Trapezoidal', alignment='center', fontsize=title_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=4 , ny=4))

    # add cross Sections
    cross_section_fontsize = 50
    ax = fig.subplots()
    plot_rectangular_cross_section(ax, status=status, fontsize=cross_section_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=0 , ny=2))
    
    ax = fig.subplots()
    plot_parabolic_cross_section(ax, status=status, fontsize=cross_section_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=2 , ny=2))
    
    ax = fig.subplots()
    plot_trapezoidal_cross_section(ax, status=status, fontsize=cross_section_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=4 , ny=2))

    # add equations
    equation_fontsize = 50
    ax = fig.subplots()
    if status > 0:
        equation_text = r'$S = w_k ~ (s_k - b_k)$'
    else:
        equation_text = ''
    set_text(ax, equation_text, alignment='center', fontsize=equation_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=0 , ny=0))

    ax = fig.subplots()
    if status > 0:
        equation_text = r'$S = 2/3~ w_k ~ (s_k - b_k)$'
    else:
        equation_text = ''
    set_text(ax, equation_text, alignment='center', fontsize=equation_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=2 , ny=0))
    
    ax = fig.subplots()
    if status > 0:
        equation_text = r'$S = (w_k + w0_k) ~ (s_k - b_k)~/~2$'
    else:
        equation_text = ''
    set_text(ax, equation_text, alignment='center', fontsize=equation_fontsize)
    ax.set_axes_locator(divider.new_locator(nx=4 , ny=0))
    
    fig.savefig(filename,format='pdf',bbox_inches='tight',dpi=300)

create_cross_section_figure('all_cross_sections_1.pdf', status=1)

# But all together

## below

In [ ]:
fontsize = 35
facecolor = 'white'

fig = plt.figure(figsize=(1, 1), facecolor=facecolor)

# define grid for axis

#       real glacier
#------------------------
#      numeric glacier
#------------------------
# Rectangular | Parabolic

# define heights
real_glacier_height = 15
separation_real_numeric_glacier = 0.2
numeric_glacier_height = 8
separation_numeric_cross_section = 0.2
cross_section_height = 8
numeration_height = 1.

#define widths
cross_section_width = 8
separation_cross_section = 1

# fixed size in inch
# along x axis                                                              x-index for locator
horiz = [Size.Fixed(cross_section_width),                                    # 0 first cross section
         Size.Fixed(separation_cross_section),   
         Size.Fixed(cross_section_width),                                    # 2 second cross section
         Size.Fixed(separation_cross_section),   
         Size.Fixed(cross_section_width),                                    # 4 third cross section
        ]

                                                                          # y-index for locator
vert = [Size.Fixed(cross_section_height),                                 # 0 cross section
        Size.Fixed(numeration_height),                                    # 1 numeration cross sections
        Size.Fixed(separation_numeric_cross_section),
        Size.Fixed(numeric_glacier_height),                               # 3 numeric glacier
        Size.Fixed(numeration_height),                                    # 4 numeration numeric glacier
        Size.Fixed(separation_real_numeric_glacier),
        Size.Fixed(real_glacier_height),                                  # 6 real glacier
        Size.Fixed(numeration_height)                                     # 7 numeration real glacier
       ]

rect = (0., 0., 1., 1.)  # Position of the grid in the figure

# divide the axes rectangle into grid whose size is specified by horiz * vert
divider = Divider(fig, rect, horiz, vert, aspect=False)

# add real glacier with flowline
ax = fig.subplots()
plot_real_glacier(ax, head_marker_size = 600, tail_marker_size = 600,
                  numeric_grid_marker_size = 400)
ax.set_axes_locator(divider.new_locator(nx=0, nx1=5, ny=6))

# numeration of real glacier
ax = fig.subplots()
set_text(ax, '(a)', fontsize=fontsize, facecolor=facecolor)
ax.set_axes_locator(divider.new_locator(nx=0, nx1=5, ny=7))

# add numeric glacier
ax = fig.subplots()
plot_numeric_glacier(ax, fontsize=fontsize, head_marker_size = 250, tail_marker_size = 250,
                     numeric_grid_marker_size = 250)
ax.set_axes_locator(divider.new_locator(nx=0, nx1=5, ny=3))

# numeration of numeric glacier
ax = fig.subplots()
set_text(ax, '(b)', fontsize=fontsize, facecolor=facecolor)
ax.set_axes_locator(divider.new_locator(nx=0, nx1=5, ny=4))

# add rectangular cross section
ax = fig.subplots()
plot_rectangular_cross_section(ax, status=2, fontsize=fontsize,
                               numeric_grid_marker_size = 250)
ax.set_axes_locator(divider.new_locator(nx=0, ny=0))

# numeration rectangular cross section
ax = fig.subplots()
set_text(ax, '(c)', fontsize=fontsize, facecolor=facecolor)
ax.set_axes_locator(divider.new_locator(nx=0, ny=1))

# add parabolic cross section
ax = fig.subplots()
plot_parabolic_cross_section(ax, status=2, fontsize=fontsize,
                             numeric_grid_marker_size = 250)
ax.set_axes_locator(divider.new_locator(nx=2, ny=0))

# numeration parabolic cross section
ax = fig.subplots()
set_text(ax, '(d)', fontsize=fontsize, facecolor=facecolor)
ax.set_axes_locator(divider.new_locator(nx=2, ny=1))

# add trapezoidal cross section
ax = fig.subplots()
plot_trapezoidal_cross_section(ax, status=2, fontsize=fontsize,
                               numeric_grid_marker_size = 250)
ax.set_axes_locator(divider.new_locator(nx=4, ny=0))

# numeration trapezoidal cross section
ax = fig.subplots()
set_text(ax, '(e)', fontsize=fontsize, facecolor=facecolor)
ax.set_axes_locator(divider.new_locator(nx=4, ny=1))

fig.savefig('flowline_numeric_grid_cross_section.pdf',format='pdf',bbox_inches='tight',dpi=300)